In [5]:
options(warn=-1)
library(readr)
suppressWarnings(library(dplyr))
library(xlsx)

In [49]:
#KODA
likvidnost <- function(datum, par) {
    ime <- paste(datum, '/', par, '.csv', sep='')
    data <- read.csv(ime, header=T)
    data$date2 <- as.POSIXct(data$date,origin="1970-01-01",tz="GMT") 
    tmp <- split(data, cut(strptime(data$date2, format="%F %R"),"hour"))
    df <- data.frame(Date=as.Date(as.character()), buy=numeric(), sell=numeric(), stringsAsFactors=FALSE)
    for (ura in tmp) {
        tmpbuy <- ura[ura[, "type"] == 0,]
        tmpsell <- ura[ura[, "type"] == 1,]
        vec <- c(ura$date2[[1]], sum(tmpbuy$amount), sum(tmpsell$amount))
        df <- rbind(df, vec)
    }
    colnames(df)<- c("date","buy","sell")
    df$difference <- df$buy - df$sell
    df$date <- as.POSIXct(df$date,origin="1970-01-01",tz="GMT")
    df$daily_sum <- cumsum(df$difference)
    df <- df[1:24,]
    return(df)
}

In [50]:
#KODA
rez <- data.frame(Date=as.Date(as.character()), buy=numeric(), sell=numeric(), difference=numeric(),
                                                                              daily_sum=numeric(),stringsAsFactors=FALSE)
for (i in 13:36) {
    if (i < 31) {
        datum <- paste(as.character(i), ".09.2017", sep="")
    } else {
        i <- (i - 30)
        datum <- paste("0", as.character(i), ".10.2017", sep="")
    }
    tmp <- likvidnost(datum, 'btcusd')
    rez <- rbind(rez, tmp)
}



daily <- data.frame(Date=as.Date(as.character()), daily_sum=numeric())

for (i in 1:24) {
    vec <- c(rez$date[i*24], rez$daily_sum[i*24])
    daily <- rbind(daily, vec)
}

colnames(daily)<- c("date","difference")
daily$date <- as.POSIXct(daily$date,origin="1970-01-01",tz="GMT")



#dim(rez)
#head(rez)

#dim(daily)
#head(daily)

In [51]:
write.xlsx(rez, "indikator_urni.xlsx", sheetName = "List1")

In [63]:
prices <- read.xlsx("btcusd_price.xlsx", sheetName = "List1", header=FALSE)
prices <- prices[,1:3]
prices <- prices[nrow(prices):1,]
colnames(prices)<- c("date","open", "close")
#prices

## DNEVNI IZRAČUN

In [64]:
daily$price_diff <- prices$close - prices$open
colnames(daily) <- c("date", "flow_difference", "price_difference")
daily$date <- substring(as.character(daily$date),1,10)
daily

date,flow_difference,price_difference
2017-09-13,-3422.69803,-249.39
2017-09-14,-3142.75894,-720.42
2017-09-15,1380.82588,471.22
2017-09-16,153.80432,-12.71
2017-09-17,50.40402,-23.40
2017-09-18,2180.96892,474.11
2017-09-19,-1179.95979,-148.82
2017-09-20,-1081.33217,-10.41
2017-09-21,-1580.77906,-270.43
2017-09-22,656.36138,2.68


In [53]:
cor(daily$flow_difference, daily$price_difference)

[1] 0.7860662

In [68]:
write.xlsx(daily, "indikator_dnevni.xlsx", sheetName = "List1", row.names=FALSE)

## URNI IZRAČUN

In [52]:
hourly <- read.xlsx("indikator_urni.xlsx", sheetName = "List1", header=TRUE, rowIndex=TRUE) 
#pazi v zgornji vrstici, da berem pravilno indikator_urni.xlsx
hourly_prices <- read.csv("Bitstamp_BTCUSD_1h.csv", header=TRUE, sep=",", skip=1)
hourly_prices <- hourly_prices[(5310-10*24+2):5307,]
hourly <- hourly[1:nrow(hourly_prices),]
hourly <- hourly[,2:5]
hourly_prices <- hourly_prices[rev(rownames(hourly_prices)),]
hourly$price_difference <- hourly_prices$Close - hourly_prices$Open
hourly$date <- hourly_prices$Date
hourly$buy <- round(hourly$buy, 2)
hourly$sell <- round(hourly$sell, 2)
hourly$difference <- round(hourly$difference, 2)

LEGENDA:

- date: datum
- buy: buying volume
- sell: selling volume
- difference: buying volume - selling volume
- price_difference: close price - open price

In [53]:
head(hourly, n=20)

date,buy,sell,difference,price_difference
2017-09-13 12-AM,227.13,343.89,-116.76,-34.30
2017-09-13 01-AM,116.48,174.50,-58.02,-33.13
2017-09-13 02-AM,321.56,711.28,-389.72,-79.11
2017-09-13 03-AM,700.25,1216.07,-515.82,-81.02
2017-09-13 04-AM,637.55,635.04,2.51,65.00
2017-09-13 05-AM,587.85,430.24,157.62,-34.01
2017-09-13 06-AM,150.23,114.79,35.45,49.02
2017-09-13 07-AM,815.44,913.30,-97.86,-77.94
2017-09-13 08-AM,418.38,593.75,-175.37,-7.18
2017-09-13 09-AM,344.71,804.21,-459.50,-10.89


Korelacija razlike v volumu in razlike  v ceni:

In [54]:
cor(hourly$difference, hourly$price_difference)

[1] 0.7793247

Povprečje razlike v volumu:

In [62]:
mean(abs(hourly$difference))

[1] 218.0038

In [55]:
colnames(hourly) <- c("date", "buy[BTC]", "sell[BTC]", "difference[BTC]", "price difference [$]")
write.xlsx(hourly, "indikator_urni.xlsx", sheetName = "List1", row.names=FALSE)

In [65]:
hourly$check <- ifelse(hourly[,"difference[BTC]"]*hourly[,"price difference [$]"]<0, 0, 1)
head(hourly)

date,buy[BTC],sell[BTC],difference[BTC],price difference [$],check
2017-09-13 12-AM,227.13,343.89,-116.76,-34.30,1
2017-09-13 01-AM,116.48,174.50,-58.02,-33.13,1
2017-09-13 02-AM,321.56,711.28,-389.72,-79.11,1
2017-09-13 03-AM,700.25,1216.07,-515.82,-81.02,1
2017-09-13 04-AM,637.55,635.04,2.51,65.00,1
2017-09-13 05-AM,587.85,430.24,157.62,-34.01,0


Ujemanje smeri gibanja razlike volumna in razlike cene v procentih:

In [69]:
sum(hourly$check) 
sum(hourly$check) / nrow(hourly)

[1] 190

[1] 0.8050847

Vsi zgornji parametri so spodaj poračunani, še za "močne signale", tj. urne vnose, ko je bila absolutna razlika volumna večja od 270:

In [80]:
hourly_extremes <- hourly[abs(hourly[,"difference[BTC]"])>270,]
head(hourly_extremes)

,date,buy[BTC],sell[BTC],difference[BTC],price difference [$],check
3,2017-09-13 02-AM,321.56,711.28,-389.72,-79.11,1
4,2017-09-13 03-AM,700.25,1216.07,-515.82,-81.02,1
10,2017-09-13 09-AM,344.71,804.21,-459.50,-10.89,1
11,2017-09-13 10-AM,361.92,1006.08,-644.16,-80.65,1
12,2017-09-13 11-AM,574.84,922.54,-347.70,-18.35,1
15,2017-09-13 02-PM,887.91,2595.37,-1707.47,-66.72,1


In [81]:
cor(hourly_extremes[,"difference[BTC]"], hourly_extremes[,"price difference [$]"])

[1] 0.8754774

In [82]:
sum(hourly_extremes$check) 
sum(hourly_extremes$check) / nrow(hourly_extremes)

[1] 64

[1] 1